## Set up the environment

Let us import all the necessary libraries

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

from pyomo.environ import *

SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

     |████████████████████████████████| 9.4MB 6.5MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 256kB 64.6MB/s 
     |████████████████████████████████| 163kB 65.7MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-

## Build the Second Model and Re-analyze

### 1. Import the data and extract the needed values

This step is the same as the one we did in the first model.

We will read the data again from the excel file.

In [ ]:
# Read data from Excel file
from google.colab import files
# Command to upload file
uploaded = files.upload()

Saving Project 1.xlsx to Project 1.xlsx


In [ ]:
import pandas as pd
import io

# Transform excel file in a pandas dataframe
df = pd.read_excel('Project 1.xlsx') 
df.head()

,Item Name,Calories,Protein (g),Total Carbs (g),Sodium (mg),Sugars (g),Weight Watchers
0,Apple Pecan Chicken Salad,350,35,27,970,17,9
1,Apple Pecan Chicken Salad Half-Size,180,18,14,490,9,5
2,Roasted Pecans,110,1,6,60,4,3
3,Pomegranate Vinaigrette Dressing,60,0,8,160,8,2
4,Baja Chili Salad,530,32,34,1580,10,15


Let us now extract the item names and other important details that we require from the file.

This process will make it easier for us to understand which item holds what number and also further in our analysis.

We have **145 items** and **6 variables** to work with here (Item name excluded).

In [ ]:
# Extract item name, nutrients and their values, and weight watchers

products = df.index.values.tolist()

products_name =  df["Item Name"].T.to_dict()

nutrients = list(df)[1:6]

nut_value = dict()
for nutrient in nutrients:
  nut_value[nutrient] = df[nutrient].tolist()

weight_watchers = df["Weight Watchers"].tolist()

products_name

{0: 'Apple Pecan Chicken Salad',
 1: 'Apple Pecan Chicken Salad Half-Size',
 2: 'Roasted Pecans',
 3: 'Pomegranate Vinaigrette Dressing',
 4: 'Baja Chili Salad',
 5: 'Baja Chili Salad Half-Size',
 6: 'Seasoned Tortilla Strips',
 7: 'Red Jalapeño Dressing',
 8: 'Chicken BLT Cobb Salad',
 9: 'Chicken BLT Cobb Salad Half-Size',
 10: 'Avocado Ranch Dressing',
 11: 'Spicy Chicken Caesar Salad',
 12: 'Spicy Chicken Caesar Salad Half-Size',
 13: 'Gourmet Croutons',
 14: 'Lemon Garlic Caesar Dressing',
 15: 'Classic Ranch Dressing',
 16: 'Light Classic Ranch Dressing',
 17: 'Italian Vinaigrette Dressing',
 18: 'Fat-Free French Dressing',
 19: 'Thousand Island Dressing',
 20: 'Garden Side Salad',
 21: 'Gourmet Croutons',
 22: 'Caesar Side Salad',
 23: 'Gourmet Croutons',
 24: 'Lemon Garlic Caesar Dressing',
 25: 'Apple Slices',
 26: 'Plain Baked Potato (avg. wgt. 10 oz.)',
 27: 'Sour Cream & Chive Baked Potato',
 28: 'Buttery Best Spread',
 29: 'Rich & Meaty Chili, Small',
 30: 'Rich & Meaty Ch

### Add new constrainsts

In our first model we had the following contraints:


*   Calories $\geq$ 2000
*   Protein $\geq$ 2000
*   Carbs $\geq$ 275
*   Sodium $\geq$ 2300
*   Sugar $\geq$ 50

Based on the drawbacks of the first model, we decide to add the limits of  the number of items products on:
* Limiting the number of each item no more than 5

* Only selecting one item form all the salad dressings and toppings.
  * Item 3, 6, 7, 10, 13~19, 21, 23, 24
* Only selecting one item from all the sauces/buttery spread
  * Item 28, 31, 39, 64~67, 99, 100, 119, 121, 125
* Only selecting two item from all beverages
  * Item 40, 58, 127
* Only selcting one item from coffee & tea
  * Item 134-144

(All numbers reference to the above product_name list)



### Build the second model

In [ ]:
# Set the model, descision variables, constraints and objective functions
model = ConcreteModel()

model.volume = Var(products,domain=NonNegativeReals,bounds = (0, 5)) # we limit that the number of each item no more than 5


obj_expr = 0
for product in products:
  obj_expr += weight_watchers[product]*model.volume[product]
model.weight_watchers = Objective(expr = obj_expr, sense = minimize)

#limit the total number of salad dressing and toppings.
model.constraint1 = Constraint(expr = model.volume[3]+model.volume[6]+model.volume[7]+model.volume[10]+model.volume[13]+
                    model.volume[14]+model.volume[15]+model.volume[16]+ model.volume[17]+model.volume[18]+
                    model.volume[19]+model.volume[21]++model.volume[23]+model.volume[24] <= 1)

#limit the total number of sauce/buttery spread
model.constraint2 = Constraint(expr = model.volume[28]+model.volume[31]+model.volume[39]+model.volume[64]+model.volume[65]+ 
                    model.volume[66] +model.volume[67]+model.volume[99]+model.volume[100]+model.volume[119]+
                    model.volume[121]+model.volume[125]  <= 1)

#limit the total number of soft drinks
model.constraint3 = Constraint(expr = model.volume[40]+model.volume[41]+model.volume[42]+model.volume[43]+model.volume[44]+
                    model.volume[45]+model.volume[46]+model.volume[47]+model.volume[48]+model.volume[49]+
                    model.volume[50]+model.volume[51]+model.volume[52]+model.volume[53]+model.volume[54]+
                    model.volume[55]+model.volume[56]+model.volume[57]+model.volume[58]+model.volume[127] <= 2)

#limit the total number of coffee and tea
model.constraint5 = Constraint(expr = model.volume[134]+model.volume[135]+model.volume[136]+model.volume[137]+
                    model.volume[138]+model.volume[139]+model.volume[140]+model.volume[141]+
                    model.volume[142]+model.volume[143]+model.volume[144] <= 1)

#Keeping the orginal constraints

model.Calories = ConstraintList()
Calories_exp = 0
for product in products:
  Calories_exp += nut_value["Calories"][product]*model.volume[product]
model.Calories.add(expr = Calories_exp >= 2000)

model.Protein = ConstraintList()
Protein_exp = 0
for product in products:
  Protein_exp += nut_value["Protein (g)"][product]*model.volume[product]
model.Protein.add(expr = Protein_exp >= 50)

model.Totalcarb = ConstraintList()
Totalcarb_exp = 0
for product in products:
  Totalcarb_exp += nut_value["Total Carbs (g)"][product]*model.volume[product]
model.Totalcarb.add(expr = Totalcarb_exp >= 275)

model.Sodium = ConstraintList()
Sodium_exp = 0
for product in products:
  Sodium_exp += nut_value["Sodium (mg)"][product]*model.volume[product]
model.Sodium.add(expr = Sodium_exp >= 2300)

model.Sugars = ConstraintList()
Sugars_exp = 0
for product in products:
  Sugars_exp += nut_value["Sugars (g)"][product]*model.volume[product]
model.Sugars.add(expr = Sugars_exp >= 50)

# show the model
model.pprint()

6 Set Declarations
    Calories_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    Protein_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    Sodium_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    Sugars_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    Totalcarb_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    volume_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  145 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21

### 4. Solve the problem and get the solution

In [ ]:
# solve it
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model).write()

solution = pd.DataFrame.from_dict(model.volume.extract_values(), orient='index', columns=[str(model.volume)])


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 48.2852348993289
  Upper bound: 48.2852348993289
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 146
  Number of nonzeros: 693
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01719212532043457
# ----------------------------------------------------------
#   Solution Information
# ----------

The selected items and its quantity from the model 2 are shown in following table.

In [ ]:
df['volume'] = solution['volume']
df[df['volume']>0]

,Item Name,Calories,Protein (g),Total Carbs (g),Sodium (mg),Sugars (g),Weight Watchers,volume
16,Light Classic Ranch Dressing,50,1,2,160,1,1,1.000000
25,Apple Slices,40,0,9,0,7,1,5.000000
26,Plain Baked Potato (avg. wgt. 10 oz.),270,7,61,25,3,7,0.811074
79,Son of Baconator®,670,38,41,1670,9,17,1.329866
100,Vanilla Frosty Waffle Cone,290,7,53,160,34,6,1.000000
127,"Orange Juice, Small",130,2,30,0,23,3,2.000000
141,"Vanilla Iced Coffee, Small",90,1,11,45,10,2,1.000000


The optimized weight watcher point is 48.285 in the model 2

In [ ]:
print("Weight watchers = ", model.weight_watchers())

sol = df[df['volume']>0]

print("Calories for the selcted items = ", sum(sol['Calories']*sol['volume']),'cal')
print("Protein for the selcted items = ", sum(sol['Protein (g)']*sol['volume']),'g')
print("Total Carbs for the selcted items = ", sum(sol['Total Carbs (g)']*sol['volume']),'g')
print("Sodium for the selcted items = ", sum(sol['Sodium (mg)']*sol['volume']),'mg')
print("Sugars for the selcted items = ", sum(sol['Sugars (g)']*sol['volume']),'g')

Weight watchers =  48.28523489932886
Calories for the selcted items =  2000.0 cal
Protein for the selcted items =  69.21241610738254 g
Total Carbs for the selcted items =  275.00000000000006 g
Sodium for the selcted items =  2606.1526845637577 mg
Sugars for the selcted items =  140.40201342281878 g


### Report sensitivity analysis

In [ ]:
# First, we will save the model (you will see the file model.lp showing up on the left after executing the line below)
model.write("/content/model.lp", io_options={'symbolic_solver_labels': True})

# After running the line below, we will generate the file "sensit.sen", which contains the report we want to see
!/usr/bin/glpsol -m /content/model.lp --lp --ranges sensit.sen

# After generating the report, we want to see its content:
!cat /content/sensit.sen

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m /content/model.lp --lp --ranges sensit.sen
Reading problem data from '/content/model.lp'...
10 rows, 146 columns, 693 non-zeros
1009 lines were read
GLPK Simplex Optimizer, v4.65
10 rows, 146 columns, 693 non-zeros
Preprocessing...
9 rows, 145 columns, 692 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.990e+03  ratio =  1.990e+03
GM: min|aij| =  1.267e-01  max|aij| =  7.892e+00  ratio =  6.229e+01
EQ: min|aij| =  1.605e-02  max|aij| =  1.000e+00  ratio =  6.229e+01
Constructing initial basis...
Size of triangular part is 9
      0: obj =   0.000000000e+00 inf =   1.263e+01 (5)
      6: obj =   5.627688517e+01 inf =   0.000e+00 (0)
*    15: obj =   4.828523490e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.2 Mb (201640 bytes)
Write sensitivity analysis report to 'sensit.sen'...
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                  

## Conclusion


Wendy's menu has a diverse set of items to choose from. When approaching the problem of how to minimize Weight Watcher points based on the FDA reccomended daily values diet the team was surprised to see the results of the first model. The optimization based on weight watcher points made the results of the first model overwhelming Minuite Maid lemonade, a ketchup packet, and one Chicken Cob Salad. Even a true lemonade fan would not be able to consume 268 servings of lemonade in one day. 

After seeing that although there were Wendy's menu items like the Minuate Maid Lemonade that only had 1 weight watcher points which naturally would be picked because of the low amount of points we had to consider the other nutrient factors of the menu items. For our second model we needed to set limitations so our consumer would have more than just one cob salad and a ton of lemonade to sustain them for a whole day. When setting up the second model and after analyzing the model the team concluded the following:

* Low weight watcher points did not equate to the most nutritious option. This was especially seen in the salad dressings and sauces that had 1-2 weight watcher points but they are not items you typically would order individually or in large quantities. In the second model we limited the selection to only one of these type of items. 
* Our research showed that the recomended amount of protein was 50g or less. The protein constraint was important for a healthy diet,  our first model did not result in sandwiches or entrees and the total protein for their daily intake was only 44g and only from one menu item, chicken cobb salad. By adding a constraint to pick a sandwhich, the other protein was fufilled in more than one other menu items chosen. For future reasearch does diversifying the protein matter? Wendy's is known for its burgers but also includes a large selection of chicken menu items.
* Sodium in processed food is something that is difficult to control. At Wendy's the menu is produced to a standardized menu and the items are all seasoned and prepared the same way. There is way to customize and ask for french fries without salt. Our second model, chose Son of Baconator which alone was 1670mg per burger, out of the FDA reccomended 2300mg. For future research, would lowering the sodium intake for the daily amount vs the FDA reccomended drive the model to be a healthier combination of menu items.
* Frosty is the iconic menu item of Wendy's. Even though it has a lot of appeal, the nutritional values of a Frosty like the Vanilla in a cone one the model chose in our model 2, one Frosty has 34 g of sugar. There is so much sugar in a Frosty that it is not recommended to have two whole Frosty's in one day as the FDA maximum is 50g of sugar per day. 
* Given the first model's result of 268 Miute Maid Lemonades that drove the team to focus on drinks in the second model. Although the constraints added were for 2 drinks and 1 coffee or tea, all cateogries for drinks contains numerous menu items with very high per drink servings of sugars. Although soda like a small Coke has 44 g of sugar, a mocha has 61 g, and Orange juice has 23g. When looking at building future models, sugary drinks could be an interesting area of focus. There are diet sodas that have zero sugar g, would this be a "better" alternative?
* Our second model also recommended a baked potato, one of the sides. Many of the sides is where the total carbs was easily getting maxed out. FDA reccomends 275 g for the daily value. A baked potao has 61 g of carbs and fries have 65g. Also, the drinks were a large source of carbs the coffee drinks were all sweetened and also containing a high amount of carbs - 40-60 per drink. It would be interesting to see if the number of drinks were not limited, would more of the carbs come from drinks as a aside for baked potato?

Overall, the second model resulted in a much more realalistc daily diet for a customer looking to eat all of their meals at Wendy's while minimizing the number of weight watcher points. Although it will be hard to truly get a low number of weight watcher points in a day without only eating a large quantity of dressings or lemonade, it is all about finding the right balance. Our team introduced informed constraints to the data selection to make it more realistic to how we as a customer would order. Assuming we typically eat 3 meals a day, which would be 3 drinks - one coffee or tea, and because we researched the importance of protein set in the number of sandwhichs so at least one made sure that the results of model 2 would have more sustenance than model 1. The team views model 2 as a better and more realistic model based on the Wendy's dataset. 